# Goni_Wind_10m

### This notebook uses [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/) to display an animated wind velocity flow field, a la https://earth.nullschool.net or  https://www.windy.com/ 

#### Original notebook made November 2020 with python == 3.7.6, xarray == 0.16.0 and ipyleaflet == 0.13.1

In [ ]:
import datetime
from ipyleaflet import Map, TileLayer, basemaps
from ipyleaflet.velocity import Velocity
import xarray as xr

### Read in a dataset containing 0-hour forecast data from the 0.25 degree GFS around the time of Goni's landfall.

In [ ]:
ds = xr.open_dataset('http://thredds.atmos.albany.edu:8080/thredds/dodsC/GRIB2/goni/goni_gfs.grib2')

In [ ]:
# Examine the dataset
ds

In [ ]:
# Select the 31 October 2020 1800 UTC analysis time and the 20m AGL level (this is the lowest level in the dataset)
u = ds['u-component_of_wind_height_above_ground'].sel(time='2020-10-31-18:00',height_above_ground1=20)
v = ds['v-component_of_wind_height_above_ground'].sel(time='2020-10-31-18:00',height_above_ground1=20)

### Create a Dataset that contains just the 20m U and V wind components, since that's what we'll pass to the `Velocity` method of ipyleaflet.
(Perhaps there is a more efficient way of doing this?)

In [ ]:
# First create the Dataset out of the 'u' Dataarray
uv = u.to_dataset()

In [ ]:
# Append v to the Dataset
uv['v'] = v

In [ ]:
# Rename the u DataArray to 'u'
name_dict = {'u-component_of_wind_height_above_ground':'u'}
uv = uv.rename_vars(name_dict)

In [ ]:
# Verify that the Dataset looks ok
uv

### Create an ipyleaflet map object; center it, specify a zoom level, and select a background map; create and add a velocity layer from the Dataset that contains u and v.

In [ ]:
center = (13.25, 124.75)
zoom = 6
m = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.CartoDB.DarkMatter)


display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}
wind = Velocity(data=uv,
                zonal_speed='u',
                meridional_speed='v',
                latitude_dimension='lat',
                longitude_dimension='lon',
                velocity_scale=0.01,
                max_velocity=120,
                display_options=display_options)
m.add_layer(wind)

### Display the map.

In [ ]:
m